# **BUILDING A CHATBOT WITH DEEP NLP**

---




## **IMPORTING THE LIBRARIES**

In [1]:
import numpy as np
import tensorflow as tf
import re
import time

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## **STEP 1: DATA PREPROCESSING**

### **IMPORTING THE DATASET**

In [3]:
path = '/content/drive/My Drive'

In [4]:
lines = open(path+'/movie_lines.txt', encoding = 'utf-8', errors = 'ignore').read().split('\n')
conversations = open(path+'/movie_conversations.txt', encoding = 'utf-8', errors = 'ignore').read().split('\n')

In [5]:
lines[0:20]

['L1045 +++$+++ u0 +++$+++ m0 +++$+++ BIANCA +++$+++ They do not!',
 'L1044 +++$+++ u2 +++$+++ m0 +++$+++ CAMERON +++$+++ They do to!',
 'L985 +++$+++ u0 +++$+++ m0 +++$+++ BIANCA +++$+++ I hope so.',
 'L984 +++$+++ u2 +++$+++ m0 +++$+++ CAMERON +++$+++ She okay?',
 "L925 +++$+++ u0 +++$+++ m0 +++$+++ BIANCA +++$+++ Let's go.",
 'L924 +++$+++ u2 +++$+++ m0 +++$+++ CAMERON +++$+++ Wow',
 "L872 +++$+++ u0 +++$+++ m0 +++$+++ BIANCA +++$+++ Okay -- you're gonna need to learn how to lie.",
 'L871 +++$+++ u2 +++$+++ m0 +++$+++ CAMERON +++$+++ No',
 'L870 +++$+++ u0 +++$+++ m0 +++$+++ BIANCA +++$+++ I\'m kidding.  You know how sometimes you just become this "persona"?  And you don\'t know how to quit?',
 'L869 +++$+++ u0 +++$+++ m0 +++$+++ BIANCA +++$+++ Like my fear of wearing pastels?',
 'L868 +++$+++ u2 +++$+++ m0 +++$+++ CAMERON +++$+++ The "real you".',
 'L867 +++$+++ u0 +++$+++ m0 +++$+++ BIANCA +++$+++ What good stuff?',
 "L866 +++$+++ u2 +++$+++ m0 +++$+++ CAMERON +++$+++ I figured yo

In [6]:
conversations[0:20]

["u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L194', 'L195', 'L196', 'L197']",
 "u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L198', 'L199']",
 "u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L200', 'L201', 'L202', 'L203']",
 "u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L204', 'L205', 'L206']",
 "u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L207', 'L208']",
 "u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L271', 'L272', 'L273', 'L274', 'L275']",
 "u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L276', 'L277']",
 "u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L280', 'L281']",
 "u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L363', 'L364']",
 "u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L365', 'L366']",
 "u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L367', 'L368']",
 "u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L401', 'L402', 'L403']",
 "u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L404', 'L405', 'L406', 'L407']",
 "u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L575', 'L576']",
 "u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L577', 'L578']",
 "u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L662', 'L663']",
 "u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L693', 'L69

### **CREATING A DICTIONARY THAT MAPS EACH LINES AND ITS ID**

In [7]:
id2line = {}

for line in lines :
  _line = line.split(' +++$+++ ') # splitting the line by the code word `+++$+++`
  if len(_line) == 5 : 
      id2line[_line[0]] = _line[4]

### **CREATING A LIST OF ALL OF THE CONVERSATIONS**

In [8]:
conversations_ids = []

for conversation in conversations[:-1] : 
  _conversation = conversation.split(' +++$+++ ')[-1][1:-1].replace("'", "").replace(" ", "")
  conversations_ids.append(_conversation.split(','))

In [9]:
conversations_ids[0:20]

[['L194', 'L195', 'L196', 'L197'],
 ['L198', 'L199'],
 ['L200', 'L201', 'L202', 'L203'],
 ['L204', 'L205', 'L206'],
 ['L207', 'L208'],
 ['L271', 'L272', 'L273', 'L274', 'L275'],
 ['L276', 'L277'],
 ['L280', 'L281'],
 ['L363', 'L364'],
 ['L365', 'L366'],
 ['L367', 'L368'],
 ['L401', 'L402', 'L403'],
 ['L404', 'L405', 'L406', 'L407'],
 ['L575', 'L576'],
 ['L577', 'L578'],
 ['L662', 'L663'],
 ['L693', 'L694', 'L695'],
 ['L696', 'L697', 'L698', 'L699'],
 ['L860', 'L861'],
 ['L862', 'L863', 'L864', 'L865']]

### **GET THE QUESTIONS AND THE ANSWERS SEPARATELY**

In [10]:
questions = []
answers = []

for conversation in conversations_ids :
  for i in range (len(conversation) - 1) : 
    questions.append(id2line[conversation[i]])
    answers.append(id2line[conversation[i+1]])

In [11]:
questions[0:20]

['Can we make this quick?  Roxanne Korrine and Andrew Barrett are having an incredibly horrendous public break- up on the quad.  Again.',
 "Well, I thought we'd start with pronunciation, if that's okay with you.",
 'Not the hacking and gagging and spitting part.  Please.',
 "You're asking me out.  That's so cute. What's your name again?",
 "No, no, it's my fault -- we didn't have a proper introduction ---",
 'Cameron.',
 "The thing is, Cameron -- I'm at the mercy of a particularly hideous breed of loser.  My sister.  I can't date until she does.",
 'Why?',
 'Unsolved mystery.  She used to be really popular when she started high school, then it was just like she got sick of it or something.',
 'Gosh, if only we could find Kat a boyfriend...',
 "C'esc ma tete. This is my head",
 "Right.  See?  You're ready for the quiz.",
 "I don't want to know how to say that though.  I want to know useful things. Like where the good stores are.  How much does champagne cost?  Stuff like Chat.  I have n

In [12]:
answers[0:20]

["Well, I thought we'd start with pronunciation, if that's okay with you.",
 'Not the hacking and gagging and spitting part.  Please.',
 "Okay... then how 'bout we try out some French cuisine.  Saturday?  Night?",
 'Forget it.',
 'Cameron.',
 "The thing is, Cameron -- I'm at the mercy of a particularly hideous breed of loser.  My sister.  I can't date until she does.",
 'Seems like she could get a date easy enough...',
 'Unsolved mystery.  She used to be really popular when she started high school, then it was just like she got sick of it or something.',
 "That's a shame.",
 'Let me see what I can do.',
 "Right.  See?  You're ready for the quiz.",
 "I don't want to know how to say that though.  I want to know useful things. Like where the good stores are.  How much does champagne cost?  Stuff like Chat.  I have never in my life had to point out my head to someone.",
 "That's because it's such a nice one.",
 'Forget French.',
 "Well, there's someone I think might be --",
 'Where?',
 "I 

### **CLEANING ALL THE TEXTS FUNCTION**

In [13]:
def clean_text(text):
    text = text.lower() # to lower case every letter in the text
    text = re.sub(r"i'm", "i am", text)
    text = re.sub(r"he's", "he is", text)
    text = re.sub(r"she's", "she is", text)
    text = re.sub(r"that's", "that is", text)
    text = re.sub(r"what's", "what is", text)
    text = re.sub(r"where's", "where is", text)
    text = re.sub(r"\'ll", " will", text)
    text = re.sub(r"\'ve", " have", text)
    text = re.sub(r"\'re", " are", text)
    text = re.sub(r"\'d", " would", text)
    text = re.sub(r"won't", "will not", text)
    text = re.sub(r"can't", "cannot", text)
    text = re.sub(r"[-()\"#/@;:<>{}+=~|.?,]", "", text)
    return text

### **CLEANING THE QUESTIONS**

In [14]:
clean_questions = []

for question in questions : 
  clean_questions.append(clean_text(question))

In [15]:
clean_questions[0:20]

['can we make this quick  roxanne korrine and andrew barrett are having an incredibly horrendous public break up on the quad  again',
 'well i thought we would start with pronunciation if that is okay with you',
 'not the hacking and gagging and spitting part  please',
 'you are asking me out  that is so cute what is your name again',
 "no no it's my fault  we didn't have a proper introduction ",
 'cameron',
 'the thing is cameron  i am at the mercy of a particularly hideous breed of loser  my sister  i cannot date until she does',
 'why',
 'unsolved mystery  she used to be really popular when she started high school then it was just like she got sick of it or something',
 'gosh if only we could find kat a boyfriend',
 "c'esc ma tete this is my head",
 'right  see  you are ready for the quiz',
 "i don't want to know how to say that though  i want to know useful things like where the good stores are  how much does champagne cost  stuff like chat  i have never in my life had to point out

### **CLEANING THE ANSWERS**

In [16]:
clean_answers = []

for answer in answers : 
  clean_answers.append(clean_text(answer))

In [17]:
clean_answers[0:20]

['well i thought we would start with pronunciation if that is okay with you',
 'not the hacking and gagging and spitting part  please',
 "okay then how 'bout we try out some french cuisine  saturday  night",
 'forget it',
 'cameron',
 'the thing is cameron  i am at the mercy of a particularly hideous breed of loser  my sister  i cannot date until she does',
 'seems like she could get a date easy enough',
 'unsolved mystery  she used to be really popular when she started high school then it was just like she got sick of it or something',
 'that is a shame',
 'let me see what i can do',
 'right  see  you are ready for the quiz',
 "i don't want to know how to say that though  i want to know useful things like where the good stores are  how much does champagne cost  stuff like chat  i have never in my life had to point out my head to someone",
 "that is because it's such a nice one",
 'forget french',
 "well there's someone i think might be ",
 'where',
 "i counted on you to help my cause 

### **REMOVING THE NON FREQUENT WORDS OCCURING BY CREATING A CORPUS**

**CREATING A DICTIONARY THAT MAPS EACH WORD TO ITS NUMBER OF OCCURANCES**

In [18]:
word2count = {}

for question in clean_questions : 
  for word in question.split() :
    if word not in word2count : 
      word2count[word] = 1
    else :
      word2count[word] += 1

for answer in clean_answers : 
  for word in answer.split() :
    if word not in word2count : 
      word2count[word] = 1
    else :
      word2count[word] += 1

### **CREATING TWO DICTIONARIES THAT MAP THE QUESTIONS WORDS AND ANSWERS WORDS TO A UNIQUE INTEGERS**

**TOKENISATION AND FILTERING THE NON FREQUENT WORDS**

In [19]:
threshold = 20
questionsWords2int = {}
word_number = 0

for word, count in word2count.items() :
  if count >= threshold : 
    questionsWords2int[word] = word_number
    word_number += 1

answersWords2int = {}   
word_number = 0

for word, count in word2count.items() :
  if count >= threshold : 
    answersWords2int[word] = word_number
    word_number += 1

**ADDING THE LAST TOKENS TO THESE DICTIONARIES**

In [20]:
tokens = ['<PAD>', '<EOS>', '<OUT>', '<SOS>']

for token in tokens:
    questionsWords2int[token] = len(questionsWords2int) + 1
for token in tokens:
    answersWords2int[token] = len(answersWords2int) + 1

### **CREATING THE INVERSE DICTIONARY OF THE ANSWERSWORDS2INT DICTIONARY**

This is to inverse map the integers in the answerWords2int dictionary to individual words.

In [21]:
answersInt2Words = {w_i : w for w, w_i in answersWords2int.items()}

### **ADDING THE `<EOS>` TOKEN TO THE END OF EVERY ANSWERS IN THE LIST**

In [22]:
for i in range (len(clean_answers)) :
  clean_answers[i] += ' <EOS>' 

In [23]:
clean_answers[0:20]

['well i thought we would start with pronunciation if that is okay with you <EOS>',
 'not the hacking and gagging and spitting part  please <EOS>',
 "okay then how 'bout we try out some french cuisine  saturday  night <EOS>",
 'forget it <EOS>',
 'cameron <EOS>',
 'the thing is cameron  i am at the mercy of a particularly hideous breed of loser  my sister  i cannot date until she does <EOS>',
 'seems like she could get a date easy enough <EOS>',
 'unsolved mystery  she used to be really popular when she started high school then it was just like she got sick of it or something <EOS>',
 'that is a shame <EOS>',
 'let me see what i can do <EOS>',
 'right  see  you are ready for the quiz <EOS>',
 "i don't want to know how to say that though  i want to know useful things like where the good stores are  how much does champagne cost  stuff like chat  i have never in my life had to point out my head to someone <EOS>",
 "that is because it's such a nice one <EOS>",
 'forget french <EOS>',
 "wel

### **TRANSLATING ALL THE QUESTIONS AND ANSWERS TO INTEGERS AND REPLACING ALL THE WORDS WHICH WERE FILTERED OUT BY `<OUT>`**

In [24]:
questions_to_int = []

for question in clean_questions :
  ints = []
  for word in question.split() :
    if word not in questionsWords2int :
      ints.append(questionsWords2int['<OUT>'])
    else :
      ints.append(questionsWords2int[word])
  questions_to_int.append(ints)      

In [25]:
answers_to_int = []

for answer in clean_answers :
  ints = []
  for word in answer.split() :
    if word not in answersWords2int :
      ints.append(answersWords2int['<OUT>'])
    else :
      ints.append(answersWords2int[word])
  answers_to_int.append(ints) 

### **SORTING THE QUESTIONS AND ANSWERS BY THE LENGTH OF THE QUESTIONS**

In [26]:
sorted_clean_questions = []
sorted_clean_answers = []

for length in range (1, 25) : # Min length = 1, Max length = 24
  for i in enumerate(questions_to_int) :
    if len(i[1]) == length :
      sorted_clean_questions.append(questions_to_int[i[0]])
      sorted_clean_answers.append(answers_to_int[i[0]])

In [27]:
sorted_clean_questions[0:20]

[[47],
 [62],
 [123],
 [147],
 [135],
 [39],
 [175],
 [39],
 [182],
 [183],
 [221],
 [36],
 [62],
 [135],
 [62],
 [110],
 [375],
 [132],
 [297],
 [211]]

In [28]:
sorted_clean_answers[0:20]

[[15,
  48,
  25,
  47,
  18,
  49,
  50,
  15,
  51,
  52,
  45,
  53,
  8824,
  54,
  52,
  55,
  41,
  56,
  18,
  57,
  58,
  59,
  60,
  61,
  8823],
 [8824,
  63,
  60,
  64,
  65,
  66,
  67,
  68,
  69,
  60,
  70,
  71,
  72,
  73,
  74,
  75,
  76,
  77,
  60,
  78,
  79,
  52,
  74,
  80,
  81,
  8823],
 [102, 8823],
 [1529, 77, 101, 1550, 33, 149, 608, 8823],
 [27, 153, 227, 3, 6453, 8823],
 [26, 27, 7, 160, 253, 65, 1280, 97, 65, 613, 8823],
 [1387, 134, 8823],
 [27, 239, 133, 194, 226, 74, 8823],
 [196, 8823],
 [20, 27, 124, 612, 32, 45, 1512, 47, 8823],
 [41,
  2582,
  157,
  18,
  44,
  78,
  45,
  103,
  1112,
  50,
  963,
  15,
  8824,
  144,
  219,
  519,
  8823],
 [111, 8824, 93, 45, 271, 8823],
 [180, 75, 77, 45, 272, 273, 8823],
 [279,
  24,
  18,
  280,
  18,
  20,
  110,
  128,
  281,
  76,
  116,
  278,
  267,
  75,
  264,
  74,
  5,
  18,
  282,
  283,
  284,
  93,
  8824,
  152,
  5,
  41,
  8824,
  8824,
  285,
  8823],
 [116, 60, 244, 4625, 383, 297, 8823],